# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

In [1]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv")
df.head()

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN


In [3]:
df.columns

Index(['Unnamed: 0', 'Customer', 'State', 'Customer Lifetime Value',
       'Response', 'Coverage', 'Education', 'Effective To Date',
       'EmploymentStatus', 'Gender', 'Income', 'Location Code',
       'Marital Status', 'Monthly Premium Auto', 'Months Since Last Claim',
       'Months Since Policy Inception', 'Number of Open Complaints',
       'Number of Policies', 'Policy Type', 'Policy', 'Renew Offer Type',
       'Sales Channel', 'Total Claim Amount', 'Vehicle Class', 'Vehicle Size',
       'Vehicle Type'],
      dtype='object')

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [5]:
new_df = df[(df["Response"] == "Yes") & (df["Total Claim Amount"] > 1000)]
new_df

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
189,189,OK31456,California,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,...,0.0,1,Corporate Auto,Corporate L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,NaN
236,236,YJ16163,Oregon,11009.130490,Yes,Premium,Bachelor,1/24/11,Employed,F,...,0.0,1,Special Auto,Special L3,Offer2,Agent,1358.400000,Luxury Car,Medsize,A
419,419,GW43195,Oregon,25807.063000,Yes,Extended,College,2/13/11,Employed,F,...,1.0,2,Personal Auto,Personal L2,Offer1,Branch,1027.200000,Luxury Car,Small,A
442,442,IP94270,Arizona,13736.132500,Yes,Premium,Master,2/13/11,Disabled,F,...,0.0,8,Personal Auto,Personal L2,Offer1,Web,1261.319869,SUV,Medsize,A
587,587,FJ28407,California,5619.689084,Yes,Premium,High School or Below,1/26/11,Unemployed,M,...,0.0,1,Personal Auto,Personal L1,Offer2,Web,1027.000029,SUV,Medsize,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10351,10351,FN44127,Oregon,3508.569533,Yes,Extended,College,1/5/11,Medical Leave,M,...,1.0,1,Personal Auto,Personal L2,Offer2,Branch,1176.278800,Four-Door Car,Small,NaN
10373,10373,XZ64172,Oregon,10963.957230,Yes,Premium,High School or Below,2/8/11,Employed,M,...,0.0,1,Corporate Auto,Corporate L2,Offer1,Agent,1324.800000,Luxury SUV,Medsize,NaN
10487,10487,IX60941,Oregon,3508.569533,Yes,Extended,College,1/5/11,Medical Leave,M,...,1.0,1,Personal Auto,Personal L3,Offer2,Branch,1176.278800,Four-Door Car,Small,NaN
10565,10565,QO62792,Oregon,7840.165778,Yes,Extended,College,1/14/11,Employed,M,...,2.0,1,Personal Auto,Personal L3,Offer2,Agent,1008.000000,NaN,NaN,NaN


2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [7]:
condition = df["Response"] == "Yes"
answer_yes = df[condition].copy()
answer_yes.groupby(["Gender", "Policy Type"])["Total Claim Amount"].mean()

Gender  Policy Type   
F       Corporate Auto    433.738499
        Personal Auto     452.965929
        Special Auto      453.280164
M       Corporate Auto    408.582459
        Personal Auto     457.010178
        Special Auto      429.527942
Name: Total Claim Amount, dtype: float64

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [27]:
df["State"].unique()

array(['Arizona', 'California', 'Washington', 'Oregon', nan, 'Nevada'],
      dtype=object)

In [33]:
state_counts = df["State"].value_counts()

In [37]:
states_with_more_than_500 = state_counts[state_counts > 500]
states_with_more_than_500

State
California    3552
Oregon        2909
Arizona       1937
Nevada         993
Washington     888
Name: count, dtype: int64

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [46]:
df.groupby(["Gender", "Education"])["Customer Lifetime Value"].agg(["max", "min", "median"])

max          min       median
Gender Education                                                  
F      Bachelor              73225.95652  1904.000852  5640.505303
       College               61850.18803  1898.683686  5623.611187
       Doctor                44856.11397  2395.570000  5332.462694
       High School or Below  55277.44589  2144.921535  6039.553187
       Master                51016.06704  2417.777032  5729.855012
M      Bachelor              67907.27050  1898.007675  5548.031892
       College               61134.68307  1918.119700  6005.847375
       Doctor                32677.34284  2267.604038  5577.669457
       High School or Below  83325.38119  1940.981221  6286.731006
       Master                50568.25912  2272.307310  5579.099207

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

In [45]:
df['Effective To Date'] = pd.to_datetime(df['Effective To Date'])
df['Month'] = df['Effective To Date'].dt.month
policy_counts = df.groupby(['State', 'Month'])['Number of Policies'].sum().reset_index()
policy_pivot = policy_counts.pivot_table(index='State', columns='Month', values='Number of Policies', aggfunc='sum')
print(policy_pivot)

Month          1     2
State                 
Arizona     3052  2864
California  5673  4929
Nevada      1493  1278
Oregon      4697  3969
Washington  1358  1225


6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

In [69]:
df['Effective To Date'] = pd.to_datetime(df['Effective To Date'], errors='coerce')

df['Month'] = df['Effective To Date'].dt.month

policy_counts_by_state_month = df.groupby(['State', 'Month'])['Number of Policies'].sum().reset_index()

total_policies_per_state = df.groupby('State')['Number of Policies'].sum().sort_values(ascending=False)

top_3_states = total_policies_per_state.head(3).index

top_3_state_monthly_policies = policy_counts_by_state_month[policy_counts_by_state_month['State'].isin(top_3_states)]

top_3_state_monthly_policies_pivot = top_3_state_monthly_policies.pivot_table(index='State', columns='Month', values='Number of Policies', aggfunc='sum')

top_3_state_monthly_policies_pivot


Month,1,2
State,,
Arizona,3052,2864
California,5673,4929
Oregon,4697,3969


7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [73]:
melted_df = df.melt(id_vars=['Customer'], value_vars=['Sales Channel'], var_name='Channel', value_name='Marketing Channel')

response_df = df[df['Response'] == 'Yes']

by_channel = response_df.groupby('Sales Channel')['Response'].count() / df.groupby('Sales Channel')['Response'].count()
by_channel

Sales Channel
Agent          0.190746
Branch         0.113787
Call Center    0.109786
Web            0.117141
Name: Response, dtype: float64